In [33]:
import spacy
import random
from unidecode import unidecode
import re
import json
import ast
#import pickle

In [35]:
filename = 'batata.txt'
filepath = 'C:\\Users\\92007848\\Documents\\AIproj\\AiTeste\\Resourses\\Files\\spaCyTxt\\'
TRAIN_DATA =[]
#pattern = '\(\'.*\', {\'entities\': \[\([0-9]*, [0-9]*, \'.*\'\),? ?\]}'
pattern1 = '[^\s\[\(\'].*\','
pattern2 = '{.*}'

#with open(filepath+filename) as fp:   # Unpickling
#    b = pickle.load(fp)
with open(filepath+filename, encoding="utf-8") as file:
    for line in file:
        listAux = []
        listAux.append(re.findall(pattern1,unidecode(line))[0].replace('\',',''))
        listAux.append(ast.literal_eval(re.findall(pattern2,unidecode(line))[0]))        
        treatedData = tuple(listAux)
        TRAIN_DATA.append(treatedData)     

TypeError: a bytes-like object is required, not 'str'

In [20]:
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [27]:
prdnlp = train_spacy(TRAIN_DATA, 20)

Statring iteration 0
{'ner': 4.444444239139557}
Statring iteration 1
{'ner': 4.3317039012908936}
Statring iteration 2
{'ner': 4.214937508106232}
Statring iteration 3
{'ner': 3.9672239422798157}
Statring iteration 4
{'ner': 3.6508814692497253}
Statring iteration 5
{'ner': 3.271207273006439}
Statring iteration 6
{'ner': 2.98831045627594}
Statring iteration 7
{'ner': 2.658048391342163}
Statring iteration 8
{'ner': 2.227592647075653}
Statring iteration 9
{'ner': 1.9775916188955307}
Statring iteration 10
{'ner': 1.7264760434627533}
Statring iteration 11
{'ner': 1.7325928024947643}
Statring iteration 12
{'ner': 0.9803473837673664}
Statring iteration 13
{'ner': 0.9185193553566933}
Statring iteration 14
{'ner': 0.5816536070778966}
Statring iteration 15
{'ner': 0.2925421434920281}
Statring iteration 16
{'ner': 0.112355797435157}
Statring iteration 17
{'ner': 0.03777029865887016}
Statring iteration 18
{'ner': 0.07340463803848252}
Statring iteration 19
{'ner': 0.02114808076839836}


In [ ]:
# Save our trained Model
modelfile = 'modeloTeste1'
prdnlp.to_disk(modelfile)

In [28]:
#Test your text
test_text = 'boa noite eu como batata.'
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

noite 4 9 qualquercoisa
batata 18 24 comida
. 24 25 comida
